<a href="https://colab.research.google.com/github/LizzetClifton/SUMMIT/blob/master/SUM%2BMIT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUM+MIT CNN Model

This notebook contains the training for the model, and the corresponding predictions, with the clean datasets. 

Upload "inverted_df" and "inverted_test_df" before running. 

## CNN for Numbers & Symbols

In [0]:
# Import all necessary packages
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#from zipfile import ZipFile
from sklearn.datasets import load_files
import matplotlib.pyplot as plt 
import pandas as pd
import cv2
import math
import imageio
import numpy as np
import os 
from keras.preprocessing.image import img_to_array
from PIL import Image, ImageOps
import glob
import re

In [0]:
# Additional imports
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [0]:
# read in clean datasets
train_math = pd.read_csv('inverted_df.csv')
test_math = pd.read_csv('inverted_test_df.csv')

In [0]:
# Function to turn df into tensorflow dataset
def tfDataset(math_dataset):
  tf.enable_eager_execution()

  # making a new list for columns so that they can be ints
  # and the target can be a string
  new_list = list(range(784))
  new_list.append('Target')

  # Used to be train symbols
  math_dataset.columns = new_list
  features = list(range(784))
  
  # Create tf training dataset
  dataset = (
      tf.data.Dataset.from_tensor_slices(
          (
              tf.cast(math_dataset[features].values, tf.float32),
              tf.cast(math_dataset['Target'].values, tf.int32)
          )
      )
  )

  return dataset

In [0]:
# Turn train and test dataframes into tf datasets
training_dataset = tfDataset(train_math)
testing_dataset = tfDataset(test_math)

In [0]:
# Create the class names
# These are the real class names, not the ones we mapped to numbers
class_names = ['0', '1', '2', '3', '4', '5','6','7','8','9','-','+','times','div']

In [0]:
# Check number of training and testing examples
num_train_examples = len(train_math)
num_test_examples = len(test_math)

print(num_train_examples, num_test_examples)

In [0]:
# Function will reshape each element in the tensor
# from [784] array to [28,28,1] array
def reshape(images, labels):
  images = tf.reshape(images, (28,28,1))
  return images, labels

# The map function applies the reshaping function to each element in the train
# and test datasets
training_dataset =  training_dataset.map(reshape)
testing_dataset  =  testing_dataset.map(reshape)

In [0]:
# We are reshaping and viewing 25 of the images from our dataset
plt.figure(figsize=(10,10))
i = 0

for (image, label) in testing_dataset.take(25):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap='gray')
    plt.xlabel(class_names[label])
    i += 1
plt.show()

### Build the Model

In [0]:
# Create the model
model = tf.keras.Sequential([
    # create first convolutional layer, with 3x3 filter and ReLu activation
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    # Perform max pooling with 2x2 matrix
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(14,  activation=tf.nn.softmax)
])

In [0]:
# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
BATCH_SIZE = 32
# Shuffle() will allocate a buffer of size of the number of training examples
# for picking random entries. Repeat() will re-initialize the dataset once all
# examples have been taken. Finally, batch() creates batches of the dataset with
# batch size given as batch_size which is also the length of the batches
train_dataset = training_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = testing_dataset.batch(BATCH_SIZE)

In [0]:
checkpoint_path = "folder_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only = True,
                                                verbose=1)

In [0]:
# Train the model
# .ceil() rounds number up to nearest integer
model.fit(train_dataset, epochs=5, 
          steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE), 
          callbacks = [cp_callback])

### Let's evaluate accuracy

In [0]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

In [0]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_dataset, steps = math.ceil(num_test_examples/32))
print('Restored model accuracy:', acc)

In [0]:
# Save the model
model.save("cnn.h5")

### Loading model

If you have already run the code up to here once in another session, then you should have saved a cnn.h5 file. If you already have that, you can skip the cells under "Build the Model" and "Let's evaluate accuracy" and run the cell below.

In [0]:
new_model = tf.keras.models.load_model('cnn.h5')
new_model.summary()
BATCH_SIZE = 32
test_dataset = testing_dataset.batch(BATCH_SIZE)
loss, acc = new_model.evaluate(test_dataset, steps = math.ceil(num_test_examples/32))
print('Restored model accuracy:', acc)

## Let's make some predictions

In [0]:
# Get the first prediction
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [0]:
type(test_images)

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [0]:
# Prediction probabilities for each target
predictions

A prediction is an array of 14 numbers. These describe the "confidence" of the model that the image corresponds to each of the 14 different elements in the dataset (10 digits, 4 symbols). We can see which label has the highest confidence value:

In [0]:
np.argmax(predictions[0])

So the model is most confident that this image is a 9 or `class_names[8]`. And we can check the test label to see this is correct:

In [0]:
test_labels[0]

We can graph this to look at the full set of 14 class predictions

In [0]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

Let's look at the first image, predictions, and prediction array. 

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)

Let's plot several images with their predictions. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percent (out of 100) for the predicted label. Note that it can be wrong even when very confident. 

In [0]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)

You now have a trained model and you can download the .h5 model file.